# Анализ NPS для телекоммуникационной компании

## Подключение к базе

Подключим все необходимые для работы базы

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Выгрузка данных (SQL)

Напишем код, который выгрузит все необходимые для исследования данные.

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       u.age,
       CASE 
        WHEN u.gender_segment = 1 THEN 'женщина'
        WHEN u.gender_segment = 0 THEN 'мужчина'
        ELSE 'другое'
       END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       a.title AS age_segment,
       t.title AS traffic_segment,
       ls.title AS lifetime_segment,
       u.nps_score,
       CASE 
        WHEN u.lt_day <=365 THEN 'new'
        WHEN u.lt_day >365 THEN 'old'
       END AS is_new,
       CASE 
         WHEN u.nps_score >=9 THEN 'supporters'
         WHEN u.nps_score >=7 AND u.nps_score <=8  THEN 'neutrals'
         WHEN u.nps_score <7 THEN 'critics'
       END AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS a ON u.age_gr_id=a.age_gr_id
LEFT JOIN traffic_segment AS t ON u.tr_gr_id=t.tr_gr_id
LEFT JOIN lifetime_segment AS ls ON u.lt_gr_id=ls.lt_gr_id

"""

Проверим, как выгляит полученная таблица

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,is_new,nps_group
0,A001A2,2320,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,old,supporters
1,A001WF,2344,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,old,supporters
2,A003Q7,467,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,old,supporters


Выгрузим данные в csv

In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Дашборд в Tableau

Ислледование проведено, дашборд доступен здесь [->>](https://public.tableau.com/views/Book2_13_16961354387640/Story1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)